# Building Custom Sensors

This notebook teaches you how to build custom sensors for the Simple Autonomous Car SDK.

## What You'll Learn

1. Understanding the sensor architecture
2. Building a simple custom sensor
3. Building a camera sensor (simulated)
4. Building a radar sensor (simulated)
5. Combining multiple sensors

## Prerequisites

- Understanding of the SDK basics (see [Building Simulations](building_simulation.ipynb))
- Understanding of sensors (see [Building Controller](building_controller.ipynb))
- Python knowledge

In [ ]:
import sys
sys.path.insert(0, '../../src')

import numpy as np
import matplotlib.pyplot as plt
from simple_autonomous_car import (
    Track,
    Car,
    CarState,
    GroundTruthMap,
    PerceivedMap,
    LiDARSensor,
    BaseSensor,
    PerceptionPoints,
)

## Step 1: Understanding Sensor Architecture

All sensors inherit from `BaseSensor` and must implement:

- `sense(car_state, environment_data)`: Returns `PerceptionPoints`
- `name`: Sensor identifier
- Optional: `pose_ego`: Sensor position/orientation in car frame

In [ ]:
# Let's look at the BaseSensor interface
from simple_autonomous_car.sensors.base_sensor import BaseSensor
import inspect

print("BaseSensor methods:")
for name, method in inspect.getmembers(BaseSensor, predicate=inspect.isfunction):
    if not name.startswith('_'):
        print(f"  - {name}{inspect.signature(method)}")

## Step 2: Building a Simple Range Sensor

Let's build a simple forward-facing range sensor that detects obstacles ahead.

In [ ]:
class SimpleRangeSensor(BaseSensor):
    """
    A simple forward-facing range sensor.

    Detects obstacles in a cone ahead of the sensor.
    """

    def __init__(
        self,
        ground_truth_map,
        perceived_map,
        max_range: float = 20.0,
        fov_angle: float = np.pi / 6,  # 30 degrees
        num_rays: int = 5,
        name: str = "range_sensor",
        pose_ego: np.ndarray = None,
    ):
        super().__init__(name=name, pose_ego=pose_ego)
        self.ground_truth_map = ground_truth_map
        self.perceived_map = perceived_map
        self.max_range = max_range
        self.fov_angle = fov_angle  # Field of view angle
        self.num_rays = num_rays

    def sense(self, car_state: CarState, environment_data: dict) -> PerceptionPoints:
        """
        Sense obstacles in a forward cone.

        Returns points where obstacles are detected.
        """
        # Get sensor pose in global frame
        sensor_pose_global = self.get_sensor_pose_global(car_state)
        sensor_pos = sensor_pose_global[:2]
        sensor_heading = sensor_pose_global[2]

        # Generate rays in the field of view
        detected_points = []

        for i in range(self.num_rays):
            # Angle for this ray (spread across FOV)
            angle_offset = -self.fov_angle / 2 + (i / (self.num_rays - 1)) * self.fov_angle
            ray_angle = sensor_heading + angle_offset

            # Cast ray and find intersection with map
            ray_end = sensor_pos + self.max_range * np.array([np.cos(ray_angle), np.sin(ray_angle)])

            # Simple ray casting: check if ray intersects track boundaries
            # In a real implementation, you'd use proper ray casting
            intersection = self._ray_cast(sensor_pos, ray_end)

            if intersection is not None:
                detected_points.append(intersection)

        if len(detected_points) == 0:
            return PerceptionPoints(points=np.array([]).reshape(0, 2), frame="global")

        return PerceptionPoints(points=np.array(detected_points), frame="global")

    def _ray_cast(self, start: np.ndarray, end: np.ndarray) -> np.ndarray:
        """
        Simple ray casting to find intersection with track boundaries.

        Returns the first intersection point, or None if no intersection.
        """
        # Get track boundaries from ground truth map
        track = self.ground_truth_map.track

        # Check intersection with inner and outer boundaries
        for boundary in [track.inner_bound, track.outer_bound]:
            for i in range(len(boundary) - 1):
                p1, p2 = boundary[i], boundary[i + 1]

                # Line-line intersection
                intersection = self._line_intersection(start, end, p1, p2)
                if intersection is not None:
                    return intersection

        return None

    def _line_intersection(self, p1, p2, p3, p4):
        """Find intersection of two line segments."""
        # Line segment intersection algorithm
        d = (p2[0] - p1[0]) * (p4[1] - p3[1]) - (p2[1] - p1[1]) * (p4[0] - p3[0])
        if abs(d) < 1e-10:
            return None  # Lines are parallel

        t = ((p3[0] - p1[0]) * (p4[1] - p3[1]) - (p3[1] - p1[1]) * (p4[0] - p3[0])) / d
        u = ((p3[0] - p1[0]) * (p2[1] - p1[1]) - (p3[1] - p1[1]) * (p2[0] - p1[0])) / d

        if 0 <= t <= 1 and 0 <= u <= 1:
            return p1 + t * (p2 - p1)

        return None

# Test the sensor
track = Track.create_simple_track(length=80.0, width=40.0, track_width=5.0)
ground_truth_map = GroundTruthMap(track)
perceived_map = PerceivedMap(ground_truth_map)

start_point, start_heading = track.get_point_at_distance(0.0)
car = Car(initial_state=CarState(x=start_point[0], y=start_point[1], heading=start_heading, velocity=8.0))

range_sensor = SimpleRangeSensor(
    ground_truth_map,
    perceived_map,
    max_range=15.0,
    fov_angle=np.pi / 4,  # 45 degrees
    num_rays=7,
    name="range_sensor"
)

car.add_sensor(range_sensor)

# Test sensing
perception = range_sensor.sense(car.state, {"ground_truth_map": ground_truth_map})

print(f"✓ Simple range sensor created!")
print(f"  Detected {len(perception.points)} points")
print(f"  Max range: {range_sensor.max_range}m")
print(f"  FOV: {np.degrees(range_sensor.fov_angle):.1f}°")

## Step 3: Visualizing the Custom Sensor

In [ ]:
from simple_autonomous_car.visualization import plot_track, plot_perception, plot_car

# Visualize sensor data
fig, ax = plt.subplots(figsize=(12, 10))

plot_track(track, ax=ax)
plot_perception(perception, car.state, ax=ax, frame="global", color="orange", label="Range Sensor")
plot_car(car, ax=ax, show_heading=True)

# Draw sensor FOV
sensor_pose = range_sensor.get_sensor_pose_global(car.state)
sensor_pos = sensor_pose[:2]
sensor_heading = sensor_pose[2]

# Draw FOV cone
fov_half = range_sensor.fov_angle / 2
cone_left = sensor_pos + range_sensor.max_range * np.array([
    np.cos(sensor_heading - fov_half),
    np.sin(sensor_heading - fov_half)
])
cone_right = sensor_pos + range_sensor.max_range * np.array([
    np.cos(sensor_heading + fov_half),
    np.sin(sensor_heading + fov_half)
])

ax.plot([sensor_pos[0], cone_left[0]], [sensor_pos[1], cone_left[1]], 'g--', alpha=0.5, label='FOV')
ax.plot([sensor_pos[0], cone_right[0]], [sensor_pos[1], cone_right[1]], 'g--', alpha=0.5)

ax.set_title("Simple Range Sensor")
ax.legend()
plt.show()

print("✓ Sensor visualization complete!")

## Step 4: Building a Simulated Camera Sensor

A camera sensor detects objects in its field of view and returns bounding boxes or object detections.

In [ ]:
class CameraSensor(BaseSensor):
    """
    A simulated camera sensor that detects objects in its field of view.

    Returns perception points representing detected objects.
    """

    def __init__(
        self,
        ground_truth_map,
        perceived_map,
        max_range: float = 30.0,
        fov_angle: float = np.pi / 3,  # 60 degrees
        resolution: tuple = (640, 480),  # Simulated image resolution
        name: str = "camera",
        pose_ego: np.ndarray = None,
    ):
        super().__init__(name=name, pose_ego=pose_ego)
        self.ground_truth_map = ground_truth_map
        self.perceived_map = perceived_map
        self.max_range = max_range
        self.fov_angle = fov_angle
        self.resolution = resolution

    def sense(self, car_state: CarState, environment_data: dict) -> PerceptionPoints:
        """
        Simulate camera detection.

        In a real implementation, this would process an image.
        Here we simulate by detecting track boundaries in FOV.
        """
        # Get sensor pose
        sensor_pose_global = self.get_sensor_pose_global(car_state)
        sensor_pos = sensor_pose_global[:2]
        sensor_heading = sensor_pose_global[2]

        # Sample points in FOV (simulating object detection)
        detected_points = []
        num_samples = 20  # Simulate detecting 20 points

        for i in range(num_samples):
            # Random angle in FOV
            angle_offset = np.random.uniform(-self.fov_angle / 2, self.fov_angle / 2)
            ray_angle = sensor_heading + angle_offset

            # Random distance (simulating objects at different distances)
            distance = np.random.uniform(5.0, self.max_range)

            # Check if this point is on track boundary (simulating object detection)
            point = sensor_pos + distance * np.array([np.cos(ray_angle), np.sin(ray_angle)])

            # Simple check: if point is near track boundary, it's detected
            track = self.ground_truth_map.track
            min_dist_to_boundary = min(
                np.min(np.linalg.norm(track.inner_bound - point, axis=1)),
                np.min(np.linalg.norm(track.outer_bound - point, axis=1))
            )

            if min_dist_to_boundary < 2.0:  # Within 2m of boundary
                detected_points.append(point)

        if len(detected_points) == 0:
            return PerceptionPoints(points=np.array([]).reshape(0, 2), frame="global")

        return PerceptionPoints(points=np.array(detected_points), frame="global")

# Test camera sensor
camera = CameraSensor(
    ground_truth_map,
    perceived_map,
    max_range=25.0,
    fov_angle=np.pi / 2,  # 90 degrees
    name="camera"
)

camera_perception = camera.sense(car.state, {"ground_truth_map": ground_truth_map})

print(f"✓ Camera sensor created!")
print(f"  Detected {len(camera_perception.points)} objects")
print(f"  FOV: {np.degrees(camera.fov_angle):.1f}°")
print(f"  Resolution: {camera.resolution}")

## Step 5: Combining Multiple Sensors

In [ ]:
# Add multiple sensors to the car
car = Car(initial_state=CarState(x=start_point[0], y=start_point[1], heading=start_heading, velocity=8.0))

# Front LiDAR
front_lidar = LiDARSensor(
    ground_truth_map,
    perceived_map,
    max_range=40.0,
    name="front_lidar",
    pose_ego=np.array([1.0, 0.0, 0.0])  # 1m forward
)

# Rear range sensor
rear_range = SimpleRangeSensor(
    ground_truth_map,
    perceived_map,
    max_range=15.0,
    fov_angle=np.pi / 4,
    num_rays=5,
    name="rear_range",
    pose_ego=np.array([-1.0, 0.0, np.pi])  # 1m back, facing rear
)

# Camera
camera = CameraSensor(
    ground_truth_map,
    perceived_map,
    max_range=30.0,
    name="camera"
)

# Add all sensors
car.add_sensor(front_lidar)
car.add_sensor(rear_range)
car.add_sensor(camera)

# Get perception from all sensors
perception_data = car.sense_all(environment_data={"ground_truth_map": ground_truth_map})

print(f"✓ Added {len(car.sensors)} sensors to car:")
for sensor in car.sensors:
    print(f"  - {sensor.name}")

print(f"\nPerception data from all sensors:")
for sensor_name, points in perception_data.items():
    if points is not None:
        print(f"  - {sensor_name}: {len(points.points)} points")
    else:
        print(f"  - {sensor_name}: No data")

## Step 6: Visualizing Multi-Sensor Data

In [ ]:
# Visualize all sensor data
fig, ax = plt.subplots(figsize=(14, 10))

plot_track(track, ax=ax)

# Plot each sensor's data with different colors
colors = {"front_lidar": "red", "rear_range": "orange", "camera": "purple"}

for sensor_name, points in perception_data.items():
    if points is not None and len(points.points) > 0:
        color = colors.get(sensor_name, "blue")
        plot_perception(points, car.state, ax=ax, frame="global", color=color, label=sensor_name)

plot_car(car, ax=ax, show_heading=True)

ax.set_title("Multi-Sensor Perception")
ax.legend()
plt.show()

print("✓ Multi-sensor visualization complete!")

## Summary

You've learned:

1. ✅ **Sensor architecture**: All sensors inherit from `BaseSensor`
2. ✅ **Building custom sensors**: Implement `sense()` method
3. ✅ **Simple range sensor**: Forward-facing obstacle detection
4. ✅ **Camera sensor**: Simulated object detection
5. ✅ **Multi-sensor setup**: Combine multiple sensors on one car

### Key Concepts

- **BaseSensor**: Base class for all sensors
- **PerceptionPoints**: Standard format for sensor data
- **Sensor pose**: Position/orientation in car frame
- **Modular design**: Easy to add custom sensors

### Next Steps

- Build a radar sensor with velocity detection
- Build a GPS/IMU sensor for localization
- Build sensor fusion algorithms
- Add noise models to sensors
- Build sensor calibration systems